In [ ]:
pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#%tensorflow_version 2.x

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [ ]:
plt.imshow(X_train[0], cmap='gray')

**Normalizing the data**

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD

**Encoder Model**

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=[28,28]))
model.add(Dense(units=400,activation='relu'))
model.add(Dense(units=200,activation='relu'))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=50,activation='relu'))
model.add(Dense(units=25,activation='relu'))

**Decoder Model**

In [ ]:
decoder = Sequential()
decoder.add(Dense(units=50,input_shape=[25],activation='relu'))
decoder.add(Dense(units=100,activation='relu'))
decoder.add(Dense(units=200,activation='relu'))
decoder.add(Dense(units=400,activation='relu'))
decoder.add(Dense(units=784,activation='sigmoid'))
decoder.add(Reshape([28,28]))

In [ ]:
autoencoder = Sequential([model,decoder])

In [ ]:
#autoencoder.compile(loss='binary_crossentropy',optimizer=SGD(lr=1.5),metrics=['accuracy'])
autoencoder.compile(loss='mean_squared_error',optimizer=SGD(lr=1.5),metrics=['mean_squared_error'])

In [ ]:
#autoencoder.fit(X_train,X_train,epochs=5,validation_data=(X_test,X_test))
#autoencoder.fit(X_train,X_train,epochs=100,validation_data=(X_test,X_test))
#autoencoder.fit(X_train,X_train,epochs=200,validation_data=(X_test,X_test))
X_train_ec = X_train*1
X_test_ec = X_test*1
for i in range(0,3):
    autoencoder.fit(X_train,X_train_ec,epochs=100,validation_data=(X_test,X_test_ec))
    X_train_ec = X_train + (X_train - autoencoder.predict(X_train))
    X_test_ec = X_test + (X_test - autoencoder.predict(X_test))

In [ ]:
prediction = autoencoder.predict(X_test[:10])

**Orignal Image**# New Section

In [ ]:
plt.imshow(X_test[0], cmap='gray')

**Reconstructed Image after passing through an autoencoder**

In [ ]:
plt.imshow(prediction[0], cmap='gray')

In [82]:
# Lossless Recovery

# Intialization
x0 = X_test[:1]*1
y0 = autoencoder.predict(x0[:1])
xu0 = tf.cast(x0*255, tf.uint8)
yu0 = tf.cast(y0*255, tf.uint8)

# Compensation-Mask-1
x = y0*1
y = autoencoder.predict(x[:1])
ex0 = x0-x
exu0 = tf.cast(ex0*255, tf.int16)
ey0 = y0-y
eyu0 = tf.cast(ey0*255, tf.int16)
eyu = eyu0*1
for k in range(1,10+1):
    #print(k)
    #x = x+(y0-y)
    for i in range(0,28,1):
        for j in range(0,28,1):
            #compensation-mask
            #if (eyu0[0][i][j]!=0):
            if (eyu[0][i][j]!=0):
            #if (eyu[0][i][j]==0):
                x[0][i][j] = x0[0][i][j] + (y0[0][i][j]-y[0][i][j])*0.1
    y = autoencoder.predict(x[:1])
    ex = x0-x
    ey = y0-y
    exu = tf.cast(ex*255, tf.int16)            
    eyu = tf.cast(ey*255, tf.int16)            
#"""
for i in range(0,28,1):
    for j in range(0,28,1):
        sum = x[0][i][j]
        if (sum<0):
            x[0][i][j] = 0
        elif (sum>1):
            x[0][i][j] = 1
#"""

xu = tf.cast(x*255, tf.uint8)       
yu = tf.cast(y*255, tf.uint8) 
print(tf.image.psnr(xu,xu0,255))
print(tf.image.psnr(yu0,xu0,255))
print(tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28))
print(tf.image.ssim(tf.expand_dims(yu0, axis=3),tf.expand_dims(xu0, axis=3),255,28))

# Compensation-Mask-2
x = x*1
y = autoencoder.predict(x[:1])
ex0 = x0-x
exu0 = tf.cast(ex0*255, tf.int16)
ey0 = y0-y
eyu0 = tf.cast(ey0*255, tf.int16)
eyu = eyu0*1
for k in range(1,10+1):
    #print(k)
    #x = x+(y0-y)
    for i in range(0,28,1):
        for j in range(0,28,1):
            #compensation-mask
            #if (eyu0[0][i][j]!=0):
            #if (eyu[0][i][j]!=0):
            if (eyu[0][i][j]==0):
                x[0][i][j] = x0[0][i][j] + (y0[0][i][j]-y[0][i][j])*0.1
    y = autoencoder.predict(x[:1])
    ex = x0-x
    ey = y0-y
    exu = tf.cast(ex*255, tf.int16)            
    eyu = tf.cast(ey*255, tf.int16)            
#"""
for i in range(0,28,1):
    for j in range(0,28,1):
        sum = x[0][i][j]
        if (sum<0):
            x[0][i][j] = 0
        elif (sum>1):
            x[0][i][j] = 1
#"""

xu = tf.cast(x*255, tf.uint8)       
yu = tf.cast(y*255, tf.uint8) 
print(tf.image.psnr(xu,xu0,255))
print(tf.image.psnr(yu0,xu0,255))
print(tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28))
print(tf.image.ssim(tf.expand_dims(yu0, axis=3),tf.expand_dims(xu0, axis=3),255,28))


tf.Tensor(54.40225, shape=(), dtype=float32)
tf.Tensor(23.413864, shape=(), dtype=float32)
tf.Tensor([0.999962], shape=(1,), dtype=float32)
tf.Tensor([0.8174888], shape=(1,), dtype=float32)
tf.Tensor(64.76947, shape=(), dtype=float32)
tf.Tensor(23.413864, shape=(), dtype=float32)
tf.Tensor([0.9999626], shape=(1,), dtype=float32)
tf.Tensor([0.8174888], shape=(1,), dtype=float32)


In [ ]:
plt.imshow(x0[0], cmap='gray')

In [ ]:
plt.imshow(x[0], cmap='gray')

In [83]:
print(exu0)
print(exu)

tf.Tensor(
[[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0 -1 -5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  0  0  0  0  0 -1
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1
    0  

In [84]:
print(eyu0)
print(eyu)

tf.Tensor(
[[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0 -1 -1 -1 -1 -2 -1 -1  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  1  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0
    0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0
    0  